# Bond Fluctuation Model for a single chain 

In [1]:
%pylab notebook
import bfm_full as bfm
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
help(bfm)

Help on module bfm_full:

NAME
    bfm_full - bfm simulation module

CLASSES
    builtins.object
        BFMSimulator
        excludedVolumeLattice
        monomer
        validBonds
    
    class BFMSimulator(builtins.object)
     |  BFMSimulator(box_, periodicity_)
     |  
     |  class providing utilities for 3D BFM simulations:
     |  monomer container, bondset, move and apply function
     |  
     |  Methods defined here:
     |  
     |  __init__(self, box_, periodicity_)
     |      setting up simulation box:
     |      box = [boxX, boxY, boxZ] (python list of int),
     |      periodicity = [pX, pY, pZ] (python list of bools), True = is periodic, False = wall
     |      bondset (instance of class validBonds)
     |      ... and setup:
     |      empty molecules as empty list,
     |      empty lattice (boxX*boxY*boxZ) with 0 as initial value (np.array),
     |      list of moves (python list)
     |  
     |  addMonomer(self, coords, attributes)
     |      add new monom

## Single chain with excluded volume 

In [3]:
''' Aufgabe 1: Lineare Kette erstellen, Ree, Rg und Ree-Autocorrelationsfunktion berechnen '''

# lineare Kette, Rg, Re, ...
def setupLinearChain(N,box):
    '''
    create single linear chain of length N in a cubic box of size box
    Input: linear chain lenght N ( int ), box size in one dimension ( int )
    Return: instance of BFMSimulator
    '''
    chainSim = bfm.BFMSimulator( [box,box,box], [True, True, True] )
    steps = [(2,0,0),(0,2,0),(0,0,2)]
    attribute = {"fixed": False}

    for n in range(N):
        if (n > 0):
            step = steps[np.random.randint(len(steps))]
            coords = chainSim.molecules[-1].coords + step
            chainSim.addMonomer(coords,attribute)
            chainSim.connectMonomers(len(chainSim.molecules)-2,len(chainSim.molecules)-1)
        else:
            chainSim.addMonomer(np.array([0,0,0]),attribute)
    
    # sanity check before return
    chainSim.checkConfig()
            
    return chainSim

def separateMolecules(molecules):
    '''
    Find single molecules of successive linear chains
    Input: instance of molecules (list of monomers)
    Return: List of separated molecules (list of lists of monomers)
    '''
    singleMolecule = []
    separatedMolecules = []
    for m in molecules:
        if (m.idx + 1) in m.bonds:
            singleMolecule.append(m)
        else:
            singleMolecule.append(m)
            separatedMolecules.append(singleMolecule)
            singleMolecule = []
    
    return separatedMolecules

def calcRee(bfmSimulator):
    ''' 
    Sort all monomers to single molecules and calculate Ree vector for each of them
    Input: instance of BFMSimulator
    Return: List of np.arrays of Ree vectors, one for each (linear) molecule
    '''
    sepMol = separateMolecules(bfmSimulator.molecules)
            
    Ree = []
    for mol in sepMol:
        Ree.append(mol[-1].coords - mol[0].coords)
    return Ree

def calcRg2(bfmSimulator):
    ''' 
    Sort all monomers to single molecules and calculate Rg componentes in x,y,z for each of them
    Input: instance of BFMSimulator
    Return: List of np.arrays of Rg vectors, one for each (linear) molecule
    '''
    sepMol = separateMolecules(bfmSimulator.molecules)
            
    Rg2All = []
    for mol in sepMol:
        # calculate center of mass
        centerOfMass = np.zeros((3))
        for mono in mol:
            centerOfMass += mono.coords
        centerOfMass /= len(mol)
        
        # get mean squared distances to center of mass
        Rg2 = np.zeros((3))
        for mono in mol:
            diff = mono.coords - centerOfMass
            Rg2 += (diff*diff)
        Rg2 /= len(mol)
        Rg2All.append(Rg2)
        
    return Rg2All
def calcPairCor(bfmSimulator, binsize=1.):
    '''
    Calculates the pair correlation function of the whole system.
    Input: instance of BFMSimulator
    Return: Two np.arrays with first the rdf and the corresponding bins.
    '''
    mol=bfmSimulator.molecules
    vol = bfmSimulator.boxX*bfmSimulator.boxY*bfmSimulator.boxZ
    # calc all distances between monomers
    # bond length in the bfm is aroun 2.68 for an excluded volume chain and 
    # 2.714 for a ideal chain -> factor of 3 
    rdf = np.zeros(( int(len(mol)*3/binSize+1) ))
    bins= np.linspace(0,int(len(mol)*3), rdf.size )
    for mono1 in mol:
        for mono2 in mol:
            if (mono1.idx != mono2.idx):
                diff = mono1.coords - mono2.coords
                rdf[int((np.dot(diff,diff))**0.5/binsize) ]+=1
    rdf *= (vol/len(mol)**2.)
    return rdf, bins

In [4]:
# setup linear chain
N = 32
boxXYZ = 64
chainSimulator = setupLinearChain(N,boxXYZ)

In [5]:
# proof Ree, Rg, config
chainSimulator.plotConfig()
print(calcRee(chainSimulator))
print(np.sqrt(np.array(calcRg2(chainSimulator))))

<IPython.core.display.Javascript object>

[array([18, 26, 18])]
[[5.53363296 8.14108984 5.0928872 ]]


In [6]:
# autocorrelation function
def sampleReDt(N,box,time):
    ''' function to get one sample of the autocorrelation of the end to end vector of a single BFM polymer chains'''
    # setup chain
    myChain = setupLinearChain(N,box)
    # short equilibration
    myChain.performMCS(100)
    
    # initial result container
    firstRe = calcRee(myChain)[0]
    ReDt = [np.dot(firstRe,firstRe)]
    Re2 = [np.dot(firstRe,firstRe)]
    
    # run the simulations
    for i in range(time-1):
        myChain.performMCS(1)
        currentRe = calcRee(myChain)[0]
        Re2.append(np.dot(currentRe,currentRe))
        ReDt.append(np.dot(firstRe,currentRe))
        
    # average the Re2
    reMean = sum(Re2)/len(Re2)
    # return the results
    return reMean, np.array(ReDt)

In [7]:
s_time = 5000
n_sample = 10
N = 32
boxXYZ = 64

meanReDt = np.zeros(s_time)
reMean = 0.0
start=time.process_time()
for n in range(n_sample):
    reC, redtC = sampleReDt(N, boxXYZ, s_time)
    # print(reC, redtC.size)
    reMean += reC
    meanReDt += redtC
    print(n+1, reMean/(n+1), meanReDt/(n+1))

reMean /= n_sample
meanReDt /= n_sample
print(reMean, meanReDt[:4])
print("Simulation time {0:.2f}s ".format( time.process_time()-start ))

applied moves / attempted moves:
946 / 3200 = 0.295625
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
15 / 32 = 0.46875
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
15 / 32 = 0.46875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
ap

applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
3 / 32 = 0.09375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
13 / 32 = 0.40625
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
2 / 32 = 0.0625
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / a

applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
4 / 32 = 0.125
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
9 / 32 = 0.28125
applied

applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
15 / 32 = 0.46875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
3 / 32 = 0.09375
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
12 / 32 = 0.375
applied move

applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
13 / 32 = 0.40625
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
2 / 32 = 0.0625
applied mov

...

applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
6 / 32 = 0.1875
applied mov

applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
15 / 32 = 0.46875
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
9 / 32 = 0.28125
applied

applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
13 / 32 = 0.40625
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
4 / 32 = 0.125
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
3 / 32 = 0.09375
applied 

applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
12 / 32 = 0.375
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
13 / 32 = 0.40625
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
5 / 32 = 0.15625
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
4 / 32 = 0.125
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
13 / 32 = 0.40625
appl

applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
10 / 32 = 0.3125
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
9 / 32 = 0.28125
applied moves / attempted moves:
15 / 32 = 0.46875
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
11 / 32 = 0.34375
applied moves / attempted moves:
8 / 32 = 0.25
applied moves / attempted moves:
6 / 32 = 0.1875
applied moves / attempted moves:
7 / 32 = 0.21875
applied moves / attempted moves:
14 / 32 = 0.4375
applied moves / attempted moves:
14 / 32 = 0.4375
applied mov

In [8]:
figure()
#plot(np.linspace(0,s_time,s_time),meanReDt/meanReDt[0])
plot(np.linspace(0,s_time,s_time),meanReDt/reMean)
xlabel(r"$\Delta t$")
ylabel(r"$c(\Delta t)$")
show()

<IPython.core.display.Javascript object>

In [9]:
''' Aufgabe 2: Lineare Kette im Confinement (Spalt/Schlitz) '''

def setupLinChainInConfinement(N,box, periodicity):
    '''
    create single linear chain of length N in a cubic box of size box
    Input: linear chain lenght N ( int ), box size in one dimension ( int )
    Return: instance of BFMSimulator
    '''
    boxX, boxY, boxZ = box
    pX, pY, pZ = periodicity
    chainSim = bfm.BFMSimulator( [boxX, boxY, boxZ], [pX, pY, pZ] )
    
    if np.all(np.array([pX, pY, pZ])):
        steps = [(2,0,0),(0,2,0),(0,0,2)]
        start = [0,0,0]
    elif ( ( not pZ ) and ( not pY ) ):
        steps = [(2,0,0)]
        start = [0,int(boxY/2),int(boxZ/2)]
    elif ( (not pZ) and (pY) ):
        steps = [(2,0,0),(0,2,0)]
        start = [0,0,int(boxZ/2)]
    else:
        print("Warning: box periodicity {} not supported!".format(periodicity))
        
    attribute = {"fixed": False}

    for n in range(N):
        if (n > 0):
            step = steps[random.randint(len(steps))]
            coords = chainSim.molecules[-1].coords + step
            chainSim.addMonomer(coords,attribute)
            chainSim.connectMonomers(len(chainSim.molecules)-2,len(chainSim.molecules)-1)
        else:
            chainSim.addMonomer(np.array(start),attribute)
    
    # sanity check before return
    chainSim.checkConfig()
            
    return chainSim

## Polymers in a slit 

In [10]:
N = 32
box = [96, 96, 4]
period = [True, True, False]
confSim = setupLinChainInConfinement(N, box, period)
confSim.plotConfig()
confSim.performMCS(100)
confSim.plotConfig()

<IPython.core.display.Javascript object>

applied moves / attempted moves:
914 / 3200 = 0.285625


<IPython.core.display.Javascript object>

In [ ]:
# Check Ree/ Rg for different slit sizes:
period = [True,True, False]
N = 32
boxXY = 64
boxZ = [64, 32, 16, 8, 4]

n_samples = 50
meanRee = []
meanRg2 = []
meanRDF = []
meanBin = []
binSize = 1. 
start=time.process_time()
for z in boxZ:
    # setup config
    confChain = setupLinChainInConfinement(N,[boxXY, boxXY, z], period)
    # run equilibration ( 10k steps)
    confChain.performMCS(10000)
    # run long simulation to make measurements
    currentRee = np.zeros((3))
    currentRg2 = np.zeros((3))
    currentRDF = np.zeros((int(N*3/binSize+1))) 
    currentBin = np.zeros((int(N*3/binSize+1))) 
    for n in range(n_samples):
        confChain.performMCS(2000)
        currentRee += np.array(calcRee(confChain)[0])
        currentRg2 += np.array(calcRg2(confChain)[0])
        tmpRDF, tmpBin =np.array(calcPairCor(confChain))
        currentRDF += tmpRDF
        currentBin += tmpBin
    print(z,": Ree = ", currentRee/n_samples,", Rg2 = ", currentRg2/n_samples)
    meanRee.append(currentRee/n_samples)
    meanRg2.append(currentRg2/n_samples)
    meanRDF.append(currentRDF/n_samples)
    meanBin.append(currentBin/n_samples)
    
print(boxZ, meanRee, meanRg2)
data = np.vstack((boxZ,np.array(meanRee)[:,0],np.array(meanRee)[:,1],np.array(meanRee)[:,2],np.array(meanRg2)[:,0],np.array(meanRg2)[:,2],np.array(meanRg2)[:,2]))
np.savetxt("slit_data_n32.dat",data)
print("Simulation time {0:.2f}s ".format( time.process_time()-start ))

applied moves / attempted moves:
87234 / 320000 = 0.27260625
applied moves / attempted moves:
17648 / 64000 = 0.27575
applied moves / attempted moves:
16753 / 64000 = 0.261765625
applied moves / attempted moves:
17522 / 64000 = 0.27378125
applied moves / attempted moves:
17188 / 64000 = 0.2685625
applied moves / attempted moves:
17423 / 64000 = 0.272234375
applied moves / attempted moves:
17348 / 64000 = 0.2710625
applied moves / attempted moves:
17158 / 64000 = 0.26809375
applied moves / attempted moves:
17538 / 64000 = 0.27403125
applied moves / attempted moves:
17197 / 64000 = 0.268703125
applied moves / attempted moves:
17130 / 64000 = 0.26765625
applied moves / attempted moves:
17186 / 64000 = 0.26853125
applied moves / attempted moves:
17743 / 64000 = 0.277234375
applied moves / attempted moves:
16868 / 64000 = 0.2635625
applied moves / attempted moves:
17689 / 64000 = 0.276390625
applied moves / attempted moves:
17705 / 64000 = 0.276640625
applied moves / attempted moves:
17500 

In [ ]:
figure()

for i in range(5):
    plot(meanBin[i]-binSize,meanRDF[i]/N/(boxXY*boxXY*boxZ[i]), label = "boxZ="+str(boxZ[i]))
plot(meanBin[i]-binSize,1./(meanBin[i]-binSize),label = "2D:$\propto r^{-1}$" )
plot(meanBin[i]-binSize,1./(meanBin[i]-binSize)**2.,label = "1D:$\propto r^{-2}$" )
legend(loc="best")
yscale('log')
xscale('log')
xlim(1,100)
xlabel(r"$r$")
ylabel(r"$g(r)/\rho$")
show()

In [ ]:
confChain.plotConfig()

figure()
plot(boxZ,np.sqrt(np.array(meanRg2)[:,0]), label = "RgX")
plot(boxZ,np.sqrt(np.array(meanRg2)[:,1]), label = "RgY")
plot(boxZ,np.sqrt(np.array(meanRg2)[:,2]), label = "RgZ")
legend(loc="best")
xlabel(r"slit size $b_{z}$")
ylabel(r"polymer size $R_{g}$")
show()

## Polymers in a pore 

In [ ]:
# Check Ree/ Rg for different pore sizes:
period = [True, False, False]
N = 32
boxX = 64
boxYZ = [64, 32, 16, 8, 4]

n_samples = 50
meanRee = []
meanRg2 = []
meanRDF = []
meanBin = []
binSize = 1.
start=time.process_time()
for yz in boxYZ:
    # setup config
    confChain = setupLinChainInConfinement(N,[boxX, yz, yz], period)
    # run equilibration ( 10k steps)
    confChain.performMCS(10000)
    # run long simulation to make measurements
    currentRee = np.zeros((3))
    currentRg2 = np.zeros((3))
    currentRDF = np.zeros((int(N*3/binSize+1))) 
    currentBin = np.zeros((int(N*3/binSize+1))) 
    for n in range(n_samples):
        confChain.performMCS(1000)
        currentRee += np.array(calcRee(confChain)[0])
        currentRg2 += np.array(calcRg2(confChain)[0])
        meanRDF.append(currentRDF/n_samples)
        meanBin.append(currentBin/n_samples)
    print(yz,": Ree = ", currentRee/n_samples,", Rg2 = ", currentRg2/n_samples)
    meanRee.append(currentRee/n_samples)
    meanRg2.append(currentRg2/n_samples)
    meanRDF.append(currentRDF/n_samples)
    meanBin.append(currentBin/n_samples)
    
print(boxYZ, meanRee, meanRg2)
data = np.vstack((boxYZ,np.array(meanRee)[:,0],np.array(meanRee)[:,1],np.array(meanRee)[:,2],np.array(meanRg2)[:,0],np.array(meanRg2)[:,2],np.array(meanRg2)[:,2]))
np.savetxt("pore_data_n32.dat",data)
print("Simulation time {0:.2f}s ".format( time.process_time()-start ))

In [ ]:
figure()
for i in range(5):
    plot(meanBin[i]-binSize,meanRDF[i]/N*N/(boxX*boxYZ[i]*boxYZ[i]), label = "boxZ="+str(boxYZ[i]))
plot(meanBin[i]-binSize,1./(meanBin[i]-binSize),label = "2D:$\propto r^{-1}$" )
plot(meanBin[i]-binSize,1./(meanBin[i]-binSize)**2.,label = "1D:$\propto r^{-2}$" )
legend(loc="best")
yscale('log')
xscale('log')
xlim(1,100)
xlabel(r"$r$")
ylabel(r"$g(r)/\rho$")
show()

In [ ]:
figure()
plot(boxYZ,np.sqrt(np.array(meanRg2)[:,0]), label = "RgX")
plot(boxYZ,np.sqrt(np.array(meanRg2)[:,1]), label = "RgY")
plot(boxYZ,np.sqrt(np.array(meanRg2)[:,2]), label = "RgZ")
legend(loc="best")
xlabel(r"pore size $b_{yz}$")
ylabel(r"polymer size $R_{g}$")
show()